In [25]:
import time
import requests
from bs4 import BeautifulSoup
import codecs
import sys

def New_Words(new_file, old_file, verbose=False):
    with codecs.open(new_file, 'r', encoding='utf-8') as File:
        new_words = File.read().splitlines()
        File.close()

    with codecs.open(old_file, 'r', encoding='utf-8') as File:
        old_words = File.read().splitlines()
        File.close()

    w = []

    for n_word in new_words:
        new_word = n_word.replace(" ", "")
        if not new_word in w:
            if not new_word in old_words:
                w.append(n_word)
    if verbose:
        print("Words in list: " + str(len(new_words)))
        print("New words: " + str(len(w)))
    return(w)

NEW_WORDS = New_Words(r"D:\BackUP\5 English\For parse\WORDS.txt", r"D:\BackUP\5 English\For parse\OLD.txt", verbose=True)

def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

LINK = "https://wooordhunt.ru/word/"
USA = "американская транскрипция слова "
#BRITISH = "британская транскрипция слова "
words = open(r"D:\BackUP\5 English\Results\table.txt", "w", encoding="utf-8")
errors = open(r"D:\BackUP\5 English\Results\errors.txt", "w", encoding="utf-8")
old_words = open(r"D:\BackUP\5 English\Results\dictonary.txt", "w", encoding="utf-8")

bad = 0
good = 0

tot = len(NEW_WORDS)
i = 0
for x in NEW_WORDS:
    page_html = requests.get(LINK + x)
    page_text = BeautifulSoup(page_html.text, 'html.parser')
    pron1 = page_text.find('span', attrs={'class': 'transcription', 'title': USA + x})
    if pron1 is not None:
        pronuntiation = pron1.text.strip()
        pronuntiation = pronuntiation.replace("|", "")
        rus = page_text.find('span', attrs={'class': 't_inline_en'})
        if rus is not None:
            r = rus.text.strip()
            s = x + ":" + pronuntiation + ":" + r
            words.write(s + '\n')
            old_words.write(x + '\n')
            good = good + 1
        else:
            errors.write(x + '\n')
            bad = bad + 1
    else:
        errors.write(x + '\n')
        bad = bad + 1
    #time.sleep(2)
    i += 1
    progress(i, tot, status = str(i) + " - " + str(tot) + " Errors: " + str(bad) + "." + " Good: " + str(good) + ".")
     
words.close()
errors.close()
old_words.close()

print("\nBAD: {0}, GOOD: {1}".format(bad, good))

Words in list: 1625
New words: 1239
[============================================================] 100.0% ...1239 - 1239 Errors: 380. Good: 859.
BAD: 380, GOOD: 859
